In [1]:
# Order preparation for SnP

# STATUS: Incomplete
# Run-time: 10 seconds

# Dependencies:
# /zdata/pkls/*.pkl - for pickles generated by 01_snp_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7497, clientId=11) # rkv tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=11) # rkv IBG live

In [2]:
%%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties

m_maxp = 0.015     # % of max margin allowed on net liquidity per scrip to limit position risk

min_rom = 0.5      # 0.5 would be 50% return

base = 0.01        # Upper or lower base multiple for prices

max_nlvp = 0.8     # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                   # max available funds for option trades = max_nlvp * NLV - initMargin
    
#...read the account info
#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

Wall time: 1.12 s


In [13]:
#...identify Stock and Option
rights = [s.right for s in df_p.contract]
df_p['Type'] = ['Stock' if r == '0' else 'Option' for r in rights]

In [ ]:
df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte is []:
        break
        
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]

SyntaxError: 'break' outside loop (cell_name, line 19)

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained.


In [16]:
discount = [m for m in map(expPricePct, expiry)]

ValueError: time data '' does not match format '%Y%m%d  %H:%M:%S'

In [ ]:
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_p.contract)
df_p['qual_contracts'] = qual_contracts
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

In [ ]:
df_p

In [ ]:
ib.disconnect()